# Data Cleaning with Pandas

In this notebook we'll go through a few basic data cleaning steps that should be performed on all new datasets where necessary.

We'll go through the process with both the `orders` and `orderlines` datasets. You can then practice these skills by cleaning the `products` dataset yourself

In [1]:
import pandas as pd

In [2]:
# orders.csv
url = "https://drive.google.com/file/d/1Vu0q91qZw6lqhIqbjoXYvYAQTmVHh6uZ/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders = pd.read_csv(path)

# orderlines.csv
url = "https://drive.google.com/file/d/1FYhN_2AzTBFuWcfHaRuKcuCE6CWXsWtG/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines = pd.read_csv(path)

Before we begin, let's create a copy of the `orders` and `orderlines` DataFrames. This way we are sure any of our changes won't affect the original DataFrames

In [3]:
orders_df = orders.copy()

orderlines_df = orderlines.copy()

One of the best ways to begin data cleaning is by exploring using `.info()`. This will tell us:
* The shape of the DataFrame
* The names of the columns
* If there are any missing values
* The datatypes of the columns

By exploring the missing values and correcting any incorrect datatypes, we often come across inconsistencies in our data.

Beyond this, we should also have a **check for any duplicate rows**.

Let's first deal with the duplicates, as it's nice and easy, then we'll explore what `.info()` has to tell us.

## 1.&nbsp; Duplicates
We can check for duplicates using the pandas [.duplicated()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html) method.

We can then delete these rows, if we wish, using [.drop_duplicates()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)

In [30]:
# orders_df
orders_df.duplicated().sum()

0

In [8]:
# orderlines_df
orderlines_df.duplicated().sum()

0

We have no duplicate rows in either DataFrame. Easy, there is no problem to solve. Normally though, if there were some duplicates, we'd drop the extra rows.

# 2.&nbsp; `.info()`

In [9]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


* `total_paid` has 5 missing values
* `created_date` should become datetime datatype

In [10]:
orderlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


* `date` should be a datetime datatype
* `unit_price` should be a float datatype

## 3.&nbsp; Missing values

### 3.1.&nbsp; Orders
* `total_paid` has 5 missing values

In [14]:
print(f"5 missing values represents {((orders_df.total_paid.isna().sum() / orders_df.shape[0])*100).round(5)}% of the rows in our DataFrame")

5 missing values represents 0.0022% of the rows in our DataFrame


In [17]:
orders_df.shape[0]

226909

In [18]:
orders_df.total_paid.isna().sum()

5

A quick way to find out a percentage here, if you don't need to print out a sentence for yourself/students/colleagues is `.value_count(normalize=True)`

In [23]:
orders_df.total_paid.isna().value_counts(normalize=True)

False    0.999978
True     0.000022
Name: total_paid, dtype: float64

As there is such a tiny amount of missing values, we will simply delete these rows, as we have enough data without them.

In [24]:
orders_df = orders_df.loc[~orders.total_paid.isna(), :]

In [33]:
orders_df.total_paid.isna().sum()

0

In [34]:
orders_df.total_paid.notna().sum()

226904

In [25]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226904 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226904 non-null  int64  
 1   created_date  226904 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226904 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 8.7+ MB


Should you have a significant number of missing values in the future, you have a choice:
+ you can impute the values
+ you can take the values from other DataFrames, if they are present there
+ you can delete the values
+ or any number of other creative solutions

Please, always consider how much time you have on your project, and what impact your method of choice will have on your final assesment.

### 3.2.&nbsp; Orderlines
There are no missing values in `orderlines_df`

## 4.&nbsp; Datatypes

### 4.1.&nbsp; Orders
* `created_date` should become datetime datatype

In [26]:
orders_df["created_date"] = pd.to_datetime(orders_df["created_date"])

In [35]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226904 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      226904 non-null  int64         
 1   created_date  226904 non-null  datetime64[ns]
 2   total_paid    226904 non-null  float64       
 3   state         226904 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 8.7+ MB


### 4.1.&nbsp; Orderlines
* `date` should be a datetime datatype
* `unit_price` should be a float datatype

#### 4.1.1.&nbsp; `date`

In [27]:
orderlines_df["date"] = pd.to_datetime(orderlines_df["date"])

In [36]:
orderlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  object        
 6   date              293983 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 15.7+ MB


#### 4.1.2.&nbsp;`unit_price`

In [37]:
orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])

ValueError: Unable to parse string "1.137.99" at position 6

As you can see when we try to convert `unit_price` to a numerical datatype, we receive a `ValueError` telling us that pandas doesn't understand the number `1.137.99`. This is probably because numbers cannot have 2 decimal points. Let's see if there are any other numbers like this.

In [38]:
orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+").value_counts()

False    257814
True      36169
Name: unit_price, dtype: int64

Looks like over 36000 rows in `orderlines` are affected by this problem. Let's work out how much that is as a percentage of our total data.

In [39]:
two_dot_percentage = ((orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+").value_counts()[1] / orderlines_df.shape[0])*100).round(2)
print(f"The 2 dot problem represents {two_dot_percentage}% of the rows in our DataFrame")

The 2 dot problem represents 12.3% of the rows in our DataFrame


This is a bit of a tricky decision as 12.3% is a significant amount of our data... and we might even end up losing a larger portion of our data than this too. For the moment we will delete the rows as we only have 2 weeks for this project and I'd like some quick, accurate results to show. If we have time at the end, we can come back and investigate this problem further, maybe there's a solution?

Each row of `orderlines` represents a product in an order. For example, if order number 175 contained 3 seperate products, then order 175 would have 3 rows in `orderlines`, one row for each of the products. If 2 of those products have 'normal' prices (14.99, 15.85) and 1 has a price with 2 decimal points (1.137.99), we need to remove the whole order and not just the affected row. If we only remove the row with 2 decimal places then any later analysis about products and prices could be misleading.

We therefore need to find the order numbers associated with the rows that have 2 decimal points, and then remove all the associated rows.

In [40]:
two_dot_order_ids_list = orderlines_df.loc[orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+"), "id_order"]
orderlines_df = orderlines_df.loc[~orderlines_df.id_order.isin(two_dot_order_ids_list)]

In [41]:
orderlines_df.shape[0]

216250

We still have 216250 rows in orderlines to work with. This should be more than enough for our evaluation.

Now that all of the 2 decimal point prices have been removed, let's try again to convert the column `unit_price` to the correct datatype.

In [42]:
orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])

In [43]:
orderlines_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216250 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                216250 non-null  int64         
 1   id_order          216250 non-null  int64         
 2   product_id        216250 non-null  int64         
 3   product_quantity  216250 non-null  int64         
 4   sku               216250 non-null  object        
 5   unit_price        216250 non-null  float64       
 6   date              216250 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 13.2+ MB


It worked perfectly

# Challenge: Clean the `products` DataFrame
Now it's your turn. Use the lessons you learnt above and clean the products DataFrame. You don't have to copy exactly what we did. Think about the consequences of your actions, sometimes it is ok to delete rows, other times you may wish to come up with more creative solutions.

In [48]:
# products.csv
url = "https://drive.google.com/file/d/1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products = pd.read_csv(path)
products.head(15)

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364
5,APP0073,Apple Composite AV Cable iPhone and iPod white,IPhone and iPod AV Cable Dock to Composite Video.,45,420.003,0,1230
6,KIN0008,Mac Memory Kingston 1GB 667MHz DDR2 SO-DIMM,1GB RAM Mac mini and iMac (2006/07) MacBook Pr...,18.99,146.471,0,1364
7,KIN0009,Mac Memory Kingston 2GB 800MHz DDR2 SO-DIMM,2GB RAM iMac with Intel Core 2 Duo (Penryn).,36.99,274.694,0,1364
8,KIN0001-2,Mac memory Kingston 4GB (2x2GB) 667MHz DDR2 SO...,RAM 4GB (2x2GB) Mac mini and iMac (2006/07) Ma...,74,669.904,0,1364
9,APP0100,Apple Adapter Mini Display Port to VGA,Adapter Mini Display Port to VGA MacBook and M...,35,330.003,0,1325


In [46]:
# your code here
products_df=products.copy()

In [47]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


# Data Type Errors
 products_df problems found: price data type object, should be float. promo_price data type object, should be float, type data type should be int, but object.


In [50]:
products_df.shape

(19326, 7)

### Look for Duplicates

In [51]:
# your code here
products_df.duplicated().sum()

8746

there are 8746 duplicated values

In [57]:
len(products_df)

19326

before removing duplicates , there are 19326 rows in products_df dataframe

In [58]:
products_df.drop_duplicates(inplace=True)

In [59]:
products_df.shape

(10580, 7)

after removing duplicates, there are 10580 rows in products_df dataframe




# Missing values

1.   desc has missing values of 7
2.   price has missing values of 46

3.   type has missing values of 50







### Look for Missing values


In [60]:
products_df.head()

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364


In [62]:
products_df.desc.isna().sum() # 7 in desc column

7

In [63]:
products_df.price.isna().sum() #46 missing in price column

46

In [65]:
products_df.type.isna().sum() #50 missing in type column

50

####Missing value in price
checking for products when price has missing values, one pattern can be found: 45 of them has 0 in_stock, their name contains ***Celly*** and SKU starts with **CEL**

In [75]:
products_df[products_df.price.isna()]

,sku,name,desc,price,promo_price,in_stock,type
34,TWS0019,Twelve South MagicWand support Apple Magic Tra...,MagicWand for wireless keyboard and Magic Trac...,NaN,299.899,0,8696
1900,AII0008,Aiino Case MacBook Air 11 '' Transparent,MacBook Air 11-inch casing with matte finish.,NaN,22.99,0,13835403
2039,CEL0020,Celly Ambo Luxury Leather Case + iPhone 6 Case...,Cover and housing together with magnet for iPh...,NaN,399.905,0,11865403
2042,CEL0007,Celly Wallet Case with removable cover Black i...,Case Book for iPhone 6 card case type.,NaN,128.998,0,11865403
2043,CEL0012,Celly Silicone Hard Shell iPhone 6 Blue,Hard Shell Silicone iPhone 6.,NaN,4.99,0,11865403
2044,CEL0014,Celly Silicone Hard Shell iPhone 6 Amarillo,Hard Shell Silicone iPhone 6.,NaN,59.895,0,11865403
2049,CEL0015,Celly fur-lined Powerbank battery 4000mAh Black,Leather-wrapped External Battery 4000mAh for i...,NaN,239.895,0,1515
2051,CEL0018,Celly Wallet Leather Case cover Black iPhone 6,Card case with transparent protective cover fo...,NaN,294.877,0,11865403
2052,CEL0023,Celly Ambo Luxury Leather Case + Case Gold iPh...,Cover and housing together with magnet for iPh...,NaN,329.894,0,11865403
2053,CEL0025,Celly Ambo Luxury Leather Case + Case iPhone 6...,Cover and housing together with magnet for iPh...,NaN,449.878,0,11865403




####missing value in type


Looking for products when ***type*** has missing values, majority of them this pattern **SKU** starts with APP, **name** has Apple iphone, **desc** has new iPhone, and **promo_price** has two decimal point data

In [74]:
products_df[products_df.type.isna()]

,sku,name,desc,price,promo_price,in_stock,type
307,SAN0017,SanDisk Cruzer Edge USB 2.0 Flash Drive 16GB,Pendrive USB 2.0 Mac and PC.,8,59.895,0,NaN
530,SAN0026,SanDisk Extreme Cruzer 16GB USB 3.0 Flash Drive,USB 3.0 flash drive 16GB USB Mac and PC.,22,209.935,0,NaN
792,APP0672,Apple iPhone 5S 16GB Space Gray,New iPhone 5S 16G Libre (ME432Y / AB).,4.694.994,4.694.994,0,NaN
797,APP0673,Apple iPhone 5S 16GB Silver,New Free iPhone 5S 16GB (ME433Y / A).,4.090.042,4.090.042,0,NaN
798,APP0675,Apple iPhone 5S 32GB Space Gray,New Free iPhone 5S 32GB (ME435Y / A).,559,3.989.975,0,NaN
1193,APP0823,Apple iPhone 6 16GB Silver,New iPhone 6 16GB Free (MG482QL / A).,639,639.001,0,NaN
1199,APP0829,Apple iPhone 6 Plus 16GB Silver,New iPhone 6 Plus 16G Free (MGA92QL / A).,749,7.490.021,0,NaN
1200,APP0822,Apple iPhone 6 16GB Space Gray,New iPhone 6 16GB Free (MG472QL / A).,639,639.001,0,NaN
1201,APP0825,Apple iPhone 6 64GB Space Gray,New iPhone 6 64GB Free (MG4F2QL / A).,749,7.490.021,0,NaN
1202,APP0826,Apple iPhone 6 64GB Silver,New iPhone 6 64GB Free (MG4H2QL / A).,749,7.503.331,0,NaN


####missing value in desc

looking for missing values in **desc**, pattern found: majority has *open* in **name** and **promo_price** consists two decimal point values

In [76]:
products_df[products_df.desc.isna()]

,sku,name,desc,price,promo_price,in_stock,type
16126,WDT0211-A,"Open - Purple 2TB WD 35 ""PC Security Mac hard ...",NaN,107,814.659,0,1298
16128,APP1622-A,Open - Apple Smart Keyboard Pro Keyboard Folio...,NaN,1.568.206,1.568.206,0,1298
17843,PAC2334,Synology DS718 + NAS Server | 10GB RAM,NaN,566.35,5.659.896,0,12175397
18152,KAN0034-A,Open - Kanex USB-C Gigabit Ethernet Adapter Ma...,NaN,29.99,237.925,0,1298
18490,HTE0025,Hyper Pearl 1600mAh battery Mini USB Mirror an...,NaN,24.99,22.99,1,1515
18612,OTT0200,OtterBox External Battery Power Pack 20000 mAHr,NaN,79.99,56.99,1,1515
18690,HOW0001-A,Open - Honeywell thermostat Lyric zonificador ...,NaN,199.99,1.441.174,0,11905404


###Deal with missing values


In [86]:
#desc column
products_df.desc.fillna(value='NO description', inplace=True)

filles the missing value with *no description*

In [91]:
products_df.loc[products_df['sku']=='PAC2334',:] #cheking if NAN value is filles

,sku,name,desc,price,promo_price,in_stock,type
17843,PAC2334,Synology DS718 + NAS Server | 10GB RAM,NO description,566.35,5.659.896,0,12175397


##price


In [97]:
#price column
products_df.price.dropna(inplace=True)

In [108]:
products_df.price.isna().value_counts(normalize=True)

False    0.995652
True     0.004348
Name: price, dtype: float64



not taking the missing value in price column


In [134]:
products_df = products_df.loc[~products_df.price.isna(), :]

In [135]:
products_df.shape

(10157, 7)

### Check / Change Data types

In [100]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10580 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10580 non-null  object
 1   name         10580 non-null  object
 2   desc         10580 non-null  object
 3   price        10534 non-null  object
 4   promo_price  10580 non-null  object
 5   in_stock     10580 non-null  int64 
 6   type         10530 non-null  object
dtypes: int64(1), object(6)
memory usage: 661.2+ KB


*price* column is object data type not float, so attempted to make it in float data type using to_numeric

In [112]:
products_df["price"] = pd.to_numeric(products_df["price"])

ValueError: Unable to parse string "1.639.792" at position 504

didnot work because it has anomaly like 1.639.792

In [113]:
products_df.price.str.contains("\d+\.\d+\.\d+").value_counts()

False    10157
True       377
Name: price, dtype: int64

377 rows are affected with problem

In [114]:
two_dot_percentage_1 = ((products_df.price.str.contains("\d+\.\d+\.\d+").value_counts()[1] / products_df.shape[0])*100).round(2)
print(f"The 2 dot problem represents {two_dot_percentage_1}% of the rows in our product DataFrame")

The 2 dot problem represents 3.58% of the rows in our product DataFrame


In [123]:
two_dot_sku_list = products_df.loc[products_df.price.str.contains("\d+\.\d+\.\d+"), "sku"]

products_df=products_df.loc[~products_df.sku.isin(two_dot_sku_list)]


not taking. this two dot decimals value in the updated product dataframe

In [124]:
products_df.shape

(10157, 7)

converting price into numeric data

In [125]:
products_df["price"] = pd.to_numeric(products_df["price"])

In [128]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10157 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sku          10157 non-null  object 
 1   name         10157 non-null  object 
 2   desc         10157 non-null  object 
 3   price        10157 non-null  float64
 4   promo_price  10157 non-null  object 
 5   in_stock     10157 non-null  int64  
 6   type         10110 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 634.8+ KB


changing data type of *promo_price* into float

In [129]:
products_df["promo_price"] = pd.to_numeric(products_df["promo_price"])

ValueError: Unable to parse string "1.119.976" at position 49

In [130]:
products_df.promo_price.str.contains("\d+\.\d+\.\d+").value_counts()

False    5836
True     4321
Name: promo_price, dtype: int64

4321 rows are affected

In [131]:
two_dot_percentage_2 = ((products_df.promo_price.str.contains("\d+\.\d+\.\d+").value_counts()[1] / products_df.shape[0])*100).round(2)
print(f"The 2 dot problem represents {two_dot_percentage_2}% of the rows in our product DataFrame")

The 2 dot problem represents 42.54% of the rows in our product DataFrame


it would be triclky if we remove this huge amount of data in total dataframe